In [3]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt
%matplotlib inline
import _pickle
import time

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
# from alexnet_backprop import *
from alexnet_guided_bp_vanilla import *
from utils import *

In [6]:
# load test set images
test_images = open('test_set_17_227.pkl', "rb")
data_set = _pickle.load(test_images) 
actor_code = get_actor_code(data_set)[0]
reversed_actor_code = get_actor_code(data_set)[1]

# load weights
weight_fpath = 'e2e17_result3/'
weight_fname = 'end2end_weights.pkl' 
weight_fname = weight_fpath + weight_fname

In [7]:
actor_code

{0: 'Alec Baldwin',
 1: 'America Ferrera',
 2: 'Angie Harmon',
 3: 'Anne Hathaway',
 4: 'Bill Hader',
 5: 'Cheryl Hines',
 6: 'Daniel Radcliffe',
 7: 'Fran Drescher',
 8: 'Gerard Butler',
 9: 'Jennifer Aniston',
 10: 'Kristin Chenoweth',
 11: 'Lorraine Bracco',
 12: 'Matt Damon',
 13: 'Michael Vartan',
 14: 'Nicolas Cage',
 15: 'Selena Gomez',
 16: 'Steve Carell'}

In [8]:
reversed_actor_code

{'Alec Baldwin': 0,
 'America Ferrera': 1,
 'Angie Harmon': 2,
 'Anne Hathaway': 3,
 'Bill Hader': 4,
 'Cheryl Hines': 5,
 'Daniel Radcliffe': 6,
 'Fran Drescher': 7,
 'Gerard Butler': 8,
 'Jennifer Aniston': 9,
 'Kristin Chenoweth': 10,
 'Lorraine Bracco': 11,
 'Matt Damon': 12,
 'Michael Vartan': 13,
 'Nicolas Cage': 14,
 'Selena Gomez': 15,
 'Steve Carell': 16}

In [9]:
len(data_set['Angie Harmon'])

15

In [10]:
def generate_true_class_saliency(data_set, graph, sess, fname):
    
    # save saliencies in an external array
    saliencies = {}
    for actor in data_set:
        saliencies[actor] = np.zeros((0,227,227,3))
    
    
    plt.figure(dpi=70, figsize=(160, 130))
    col=0
    for actor in data_set:
        row = 0 
        for _ in range(len(data_set[actor])):
            
            # forward pass
            image = data_set[actor][_].astype(np.float64)
            image_feed = np.expand_dims(image,0)
            probabilities = sess.run(graph.probabilities, feed_dict={graph.inputs:image_feed})[0]
    
            # plot original image
            index = row*len(data_set)*2 + col +1
            plt.subplots_adjust(bottom = 0.0, wspace=0.15, hspace = 0.1, top=1.0)
            plt.subplot(len(data_set[actor]), len(data_set)*2, index)
            plt.axis('off')
            plt.title("True: " + actor + "\n Pred: " + actor_code[np.argmax(probabilities)], fontsize=14)
            plt.imshow(image.astype('uint8'))
    
            # plot true class_saliency
            i = reversed_actor_code[actor]
            one_hot = np.zeros(len(actor_code))
            one_hot[i] = 1
            plt.subplot(len(data_set[actor]), len(data_set)*2, index+1)
            plt.axis('off')
            plt.title(actor + '\n' + str(probabilities[i]), fontsize=14)
            saliency = guided_backprop(graph, image, one_hot, sess)
            plt.imshow(saliency)

            saliencies[actor] = np.vstack((saliencies[actor], np.expand_dims(saliency, axis=0)))
            
            row += 1
        col += 2
    plt.savefig(fname +'.png')
    plt.close()
    
    pickle_out = open(fname +".pkl","wb")
    _pickle.dump(saliencies, pickle_out)

In [14]:
# hyperparameters and backprop variants
tau = 1.0

In [15]:
tf.reset_default_graph()
b_graph = backprop_graph(17, 100, alexnet_face_classifier)
b_graph.classifier_graph(temp=tau)
b_graph.guided_backprop_graph()

In [16]:
with tf.Session() as sess:
    b_graph.cnn.load_weights(weight_fname, sess)
    generate_true_class_saliency(data_set, b_graph, sess, 'true_class_saliency_vanilla_temp1')

0 conv1W (11, 11, 3, 96)
1 conv1b (96,)
2 conv2W (5, 5, 48, 256)
3 conv2b (256,)
4 conv3W (3, 3, 256, 384)
5 conv3b (384,)
6 conv4W (3, 3, 192, 384)
7 conv4b (384,)
8 conv5W (3, 3, 192, 256)
9 conv5b (256,)
10 fc1W (43264, 100)
11 fc1b (100,)
12 fc2W (100, 17)
13 fc2b (17,)
